# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 1-1
## Учимся переводить между decision/evaluation/search на примере Bin Packing

**Даны:** 
* $[w_1, \dots , w_k], w_i \in [0, 1]$ — веса грузов  
* $n_{\text{bins}}$ — количество корзин грузоподъемностью 1.

Предполагается, что функция `solve_bp_decision(weights, n_bins)` решает «распознавательный» (decision) вариант задачи bin packing. На вход ей подаётся список весов и число корзин. Функция возвращает булевский ответ на вопрос «можно ли заданные веса раскидать по не более чем `n_bins` контейнерам? 

Напишите содержимое функции `solve_bp_evaluation`, которая возвращает оптимальное число корзин (решает evaluation variant задачи bin packing). 

Затем напишите содержимое функции `solve_bp_search`, которая возвращает список номеров корзин, которые при каком-нибудь оптимальном распределении присваиваются весам из списка `weights` (корзины нумеруются с единицы). 

Каждая из следующих функций должна вызывать предыдущую не более чем полиномиальное число раз.

In [56]:
#eps was added, because some tests worked incorrectly (for example [0.65, 0.3, 0.1, 0.05, 0.9])
eps = 1e-9

def solve_bp_decision(weights: list, n_bins: int) -> bool:
    def able_to_pack(weights: list, bin_capacities) -> bool:
        return weights == [] or any( 
            able_to_pack( weights[:-1], bin_capacities[:i] + [capacity - weights[-1]] + bin_capacities[(i + 1):] ) 
            for i, capacity in enumerate(bin_capacities) if (capacity - weights[-1]) > eps or abs(capacity - weights[-1]) < eps 
        )
    return able_to_pack( sorted(weights), [1.0] * n_bins )

# To test this, run:
#solve_bp_decision([0.65, 0.3, 0.1, 0.05, 0.9], 2)
# solve_bp_decision([0.8, 0.09, 0.4, 0.7], 3)

In [57]:
def solve_bp_evaluation(weights: list) -> int:
    if (len(weights) == 0):
        return 0
    l = 0
    r = len(weights)
    while (r - l > 1):
        c = (l + r) // 2
        if (solve_bp_decision(weights, c) == False):
            l = c
        else:
            r = c
    return r
    
#solve_bp_evaluation([0.8, 0.09, 0.4, 0.7])

In [58]:
p = []

def getp(ver):
    if (p[ver] == ver):
        return ver
    tans = getp(p[ver])
    p[ver] = tans
    return tans

def unite(a, b):
    a = getp(a)
    b = getp(b)
    if (a == b):
        return
    p[a] = b
    return

def solve_bp_search(weights: list):
    ret = []
    was = []
    ans = []
    gans = solve_bp_evaluation(weights)
    for i in range(len(weights)):
        p.append(i)
        ret.append(0)
        was.append(0)
        ans.append(0)
    it = 0
    for i in range(len(weights)):
        if (was[i] == 1):
            continue
        for j in range(i + 1, len(weights)):
            if (was[j] == 1):
                continue
            tmp = weights[j]
            if ((weights[i] + tmp - 1.0) > eps):
                continue
            weights[i] += tmp
            weights[j] = 0
            tans = solve_bp_evaluation(weights)
            if (tans == gans):
                unite(i, j)
                was[j] = 1
            else:
                weights[j] = tmp
                weights[i] -= tmp
    for i in range(len(weights)):
        num = getp(i)
        if (ans[num] == 0):
            it += 1
            ans[num] = it
            ret[i] = it
        else:
            ret[i] = ans[num]
    return ret

#solve_bp_search([0.65, 0.3, 0.1, 0.05, 0.9])